<a href="https://colab.research.google.com/github/mkane968/Corpus-Analysis-with-SpaCy/blob/main/notebooks/PH_Tutorial_Corpus_Analysis_with_SpaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus Analysis with spaCy (Jupyter Notebook)


## 1. Install and Import Packages

Install and import spaCy and related libraries and packages. To improve efficiency, it is common practice to do a single install at the very top of the file instead of interspersing them with your code. These packages can be run in a single cell of code; below, the markdown text describes how each downloaded package or library will be used in the analysis. 

*Note: the first time you run this code, you may need to install spaCy itself; following this, you can just retrieve the associated packages using the import function*


In [ ]:
#Imports spaCy itself, necessary to use features 
#!pip install spaCy
import spacy
#Load the natural language processing pipeline
#e'll choose eng_core_web_sm, the small English pipeline which has been tagged on written web texts
#!python -m spacy download en_core_web_md 
nlp = spacy.load("en_core_web_sm")
#Load spaCy visualizer
from spacy import displacy
#Import pandas DataFrame packages
import pandas as pd
#Import graphing package
import plotly.graph_objects as go
import plotly.express as px

Unique to Jupyter notebook, import the os module to faciliate file uploads.

In [ ]:
import os 

## 2. Load Text Files into DataFrame

After all necessary packages have been installed, it is time to upload the texts for analysis. The key here is to get the texts into a recognizeable format for analysis. 

Run the following code to change your working directory, the location where Python will search for files to process. Before completing this step, you will need to have downloaded the student papers and metadata to your local machine. Copy the name of the path to your files and insert it below. 

In [ ]:
##Get current working directory 
path = os.getcwd()
print(path)

#Change working directory
path = os.chdir("/Users/megankane/Documents/Corpora/MICUSP/MICUSP_SELECT/")

Next, upload the files for analysis. This can be done by creating a for loop which searches through the folder specified as the working directory, finding all .txt files, and appending them and their file names to two separate lists.  

In [ ]:
#Create empty lists for file names and contents
filenames = []
data = []
files = [f for f in os.listdir(path) if os.path.isfile(f)]

#Retrieve all txt files and file names and append to two lists
for f in files:
    if f.endswith('.txt'):
        with open (f, "rb") as myfile:
            filenames.append(myfile.name)
            data.append(myfile.read())

These two lists can be transformed into a dictionary, where each file name is associated with the body of text. 

In [ ]:
#Create dictionary object associating each file name with its text
d = {'Filename':filenames,'Text':data}

The dictionary can be transformed into a Pandas DataFrame. This will organize the texts into a table of rows and columns–in this case, the first column will contain the names of the files, and the second column will contain the context of each file.

In [ ]:
#Turn dictionary into a dataframe
paper_df = pd.DataFrame(d)
paper_df

Next we will add in metadata of interest to the data frame. Here, we'll add discipline and genre information connected to the MICUSP papers, as we'll be interested in using SpaCy to trace differences across genre and disciplinary categories later. Run the following code to locate the csv file (specify the path) and use it to create a Pandas dataframe.

Display the first five rows to check that the data is as expected. Four rows should be present: the paper IDs, their titles, their discipline, and their type. 

In [ ]:
metadata_df = pd.read_csv('/Users/megankane/Documents/Corpora/MICUSP/MICUSP_SELECT/metadata.csv')
metadata_df.head()

Notice that the paper ids in this DataFrame are *almost* the same as the paper file names. We're going to make them match exactly so we can merge the two DataFrames together on this column; in effect, linking each text with their title, discipline and genre. 

To match the columns, we'll remove the ".txt" tag from the end of each filename in the paper DataFrame, and then we'll rename the paper id column "Filename."

In [ ]:
#Remove .txt from title of each paper
paper_df['Filename'] = paper_df['Filename'] .map(lambda x: x.rstrip('.txt'))

#Rename column from paper ID to Title
metadata_df.rename(columns={"PAPER ID": "Filename"}, inplace=True)

Now it is possible to combine the papers and metadata into a single DataFrame. Check the first five rows to make sure each has a filename, title, discipline, paper type and text (the full paper).

In [ ]:
#Merge metadata and papers into new DataFrame
#Will only keep rows where both essay and metadata are present
final_paper_df = metadata_df.merge(paper_df,on='Filename')

#Print DataFrame
final_paper_df.head()

The resulting DataFrame is now ready for cleaning and analysis. 


## 3. Cleaning and Tokenization

From a quick scan of the DataFrame, it is evident that some preliminary cleaning is required. First use the .decode() module to remove any utf-8 characters embedded in the texts (b'\xef\xbb\xbf). It is also important to remove newline characters (\n, \r) through a simple string replacement line. These are NOT functions of spaCy but are necessary to make the code recognizable for further cleaning and tokenization. 


In [ ]:
#Remove encoding characters from Text column (b'\xef\xbb\xbf)
final_paper_df['Text'] = final_paper_df['Text'].apply(lambda x: x.decode('utf-8', errors='ignore'))
final_paper_df.head()

#Remove newline characters
final_paper_df['Text'] = final_paper_df['Text'].str.replace(r'\s+|\\r', ' ', regex=True) 
final_paper_df['Text'] = final_paper_df['Text'].str.replace(r'\s+|\\n', ' ', regex=True) 
final_paper_df.head()

The next, most basic operation to perform is lowercasing all tokens in the texts. This will prevent incorrect calculations in later case-sensitive analysis; for example, if lowercasing is not performed, “House” and “house” may be counted as two different words. 

In [ ]:
#Lowercase all words
final_paper_df['Text'] = final_paper_df['Text'].str.lower()

final_paper_df.head()

The next step is to remove punctuation. Depending on your analysis goals, you may want to keep punctuation, but in this case we are interested in words only. 

In [ ]:
#Remove punctuation and replace with no space (except periods and hyphens)
final_paper_df['Text'] = final_paper_df['Text'].str.replace(r'[^\w\-\.\'\s]+', '', regex = True)

#Remove periods and replace with space (to prevent incorrect compounds)
final_paper_df['Text'] = final_paper_df['Text'].str.replace(r'[^\w\-\'\s]+', ' ', regex = True)

final_paper_df.head()

Tokenization is the process used to split up full text into smaller parts for analysis. SpaCy has a built-in function for tokenization that involves segmenting texts into individual parts like words and punctuation. Take the example of an individual sentence: 

In [ ]:
doc = nlp("This is 'an' example? sentence")
for token in doc:
    print(token.text)

What this function is doing is calling the spaCy nlp pipeline, which contains the data and components needed for text processing. When the nlp pipeline is called on a sentence, it splits that sentence on each whitespace and reviews its components. Components are then split based on rules for words, punctuations, prefixes, suffixes, etc. Each token is then loaded into a new object that we’ve called “doc.” Calling nlp also enables part of speech tagging, lemmatization, and other enrichment procedures we’ll discuss further below. 

Since we are working with multiple long texts, we are going to use nlp.pipe, which processes batches of texts as doc objects. Here we’ll tokenize each text in our DataFrame, append each set of tokens to a list, and add the new token lists to a new column in the DataFrame.


In [ ]:
#Tokenize with spaCy
#Create list for tokens
token_list = []

# Disable POS, Dependency Parser, and NER since all we want is tokenizer 
with nlp.disable_pipes('tagger', 'parser', 'ner'):
  #Iterate through each doc object (each text in DataFrame) and tokenize, append tokens to list
    for doc in nlp.pipe(final_paper_df.Text.astype('unicode').values, batch_size=100):
        word_list = []
        for token in doc:
            word_list.append(token.text)

        token_list.append(word_list)
        
#Make token list a new column in DataFrame
final_paper_df['Tokens'] = token_list

#Check token list
final_paper_df['Tokens']

When tokenizing texts, you can also exclude stopwords. Stopwords are words which may hold little significance to text analysis, such as very common words like “the” or “and.” SpaCy has a built-in dictionary of stopwords which you can access. You can also add or remove your own stopwords, as shown below:

In [ ]:
#Adding and removing stopwords to default list
#See list of default stopwords
print(nlp.Defaults.stop_words)

#Remove a  stopword
nlp.Defaults.stop_words.remove("becomes")

#Add stopword
nlp.Defaults.stop_words.add("book")

#Check updated list of default stopwords
print(nlp.Defaults.stop_words)

To tokenize texts without stopwords, follow the same process above using nlp.pipe, but only append tokens to list that are NOT included in stopwords list and append these to a new row in the DataFrame. 


In [ ]:
#Remove all stopwords and append remaining tokens to new df column
token_list_nostops = []

# Disable POS, Dependency Parser, and NER since all we want is tokenizer 
with nlp.disable_pipes('tagger', 'parser', 'ner'):
  #Iterate through each doc object (each text in DataFrame) and tokenize, append tokens to list
    for doc in nlp.pipe(final_paper_df.Text.astype('unicode').values, batch_size=100):
        nostops_word_list = []
        for token in doc:
            if token.text not in nlp.Defaults.stop_words:
              nostops_word_list.append(token.text)

        token_list_nostops.append(nostops_word_list)

#Make token list a new column in DataFrame
final_paper_df['Tokens_NoStops'] = token_list_nostops

#Check list of tokens without stopwords
final_paper_df['Tokens_NoStops']

At this point, we have two tokenized lists: one with and without stopwords. Depending on the purposes of your analyses, you might need your data in token form. One of the benefits of spaCy, however, is that it's streamlined: when you ask it to tag parts-of-speech in a dataset, its *first* step (behind the scenes) is to tokenize the data. In other words, we don't need to run this tokenization process before running part-of-speech tagging or other analyses; we're just doing it here to demonstrate how spaCy works under-the-hood, and to get a dataset that's useful for [other types of analysis](https://towardsdatascience.com/tokenization-for-natural-language-processing-a179a891bad4). Many forms of sentiment analysis, for example, require data to be tokenized, as this form of analysis involves finding the polarity of each separate word in a document.

Even so, tokenizing the dataset without stopwords will be of value to our analysis. We can do this once, at the outset, to speed up runtime later--in essence, removing the stopwords once, so the nlp pipeline doesn't have to do the same every time. All we need to do to make the dataset with no stopwords useable for spaCy analysis is to transform each essay from a list of tokens back into a string. 

In [ ]:
final_paper_df['Text_NoStops'] = [' '.join(map(str, l)) for l in final_paper_df['Tokens_NoStops']]
final_paper_df['Text_NoStops']

Why analyze text without stopwords? One case where stopword removal may be useful is if you want to compare document similarity. SpaCy calculates document similarity based on corpus word vectors; since stopwords are words that appear throughout texts, they will heighten document similarity scores even if their content is very different. To make this type of analysis more accurate, we'll load a larger spaCy pipeline with vectors (en_core_web_md). 


In [ ]:
#Stopwords Test Case - Word Vector Similarity
#Load a larger pipeline with vectors to make stopword analysis more accurate
!spacy download en_core_web_md
nlp2 = spacy.load("en_core_web_md")

Observe the difference between these two texts, with and without stopwords. As expected, these two texts in different disciplines are less similar with stopwords removed.

In [ ]:
# Compare similarity between two documents without stopwords
doc1 = nlp2(final_paper_df.Text[0])
doc2 = nlp2(final_paper_df.Text[100])
print(f'The similarity between ' + str(final_paper_df.Filename[0]) + ' and ' + str(final_paper_df.Filename[100]) + ' with stopwords is ' + str(doc1.similarity(doc2)))

# Compare similarity between two documents with stopwords
doc1 = nlp2(final_paper_df.Text_NoStops[0])
doc2 = nlp2(final_paper_df.Text_NoStops[72])
print(f'The similarity between ' + str(final_paper_df.Filename[0]) + ' and ' + str(final_paper_df.Filename[100]) + ' without stopwords is ' + str(doc1.similarity(doc2)))


Stopword removal is also useful for topic modeling and classification tasks, where finding general themes across documents is the goal. However, other types of analysis like sentiment analysis are highly sensitive and removing stopwords will change sentence meaning (e.g. removing “not” in the sentence “I was not happy”). When possible, it is recommended to run analysis with and without stopwords and see how the model is impacted. For the rest of this tutorial, we will be using the corpus without stopwords, but you are welcome to replicate analysis with them. 

At this point, we have a cleaned DataFrame on which we can use spaCy for further text enrichment. But if you don't need lemmas, parts of speech, and named entities for your further analytic purposes, you can download the DataFrame as it is to your machine using this code. 

In [ ]:
final_paper_df.to_csv('final_paper_df.csv') 

We'll create a new DataFrame to which we'll add the spaCy enrichments (lemmas, part of speech tags, named entities). We'll only keep the filenames, discipline and paper type labels, and the text without stopwords. 

In [ ]:
#Create new DataFrame for text enrichment
enriched_df = final_paper_df[['Filename','DISCIPLINE','PAPER TYPE','Text_NoStops']].copy()
enriched_df.head()

We'll also use the smaller spaCy pipeline which is optimized for tagging, parsing, lemmatizing and ner. It's not as accurate as the en_core_web_md (and _lg) models, but processing will be faster. [Learn more about the models here](https://spacy.io/models).

## 4. Text Enrichment

### a. Lemmatization

SpaCy enables several types of text enrichment. We’ll start with lemmatization, which retrieves the dictionary root word of each word (e.g. “brighten” for “brightening”). Lemmatization is one of the functions that occurs when the nlp pipe is called; repeat the same process as above to iterate through each document in the dataframe and this time append all lemmas to new column. 


In [ ]:
#Get lemmas
lemma_list = []

# Disable Dependency Parser, and NER since all we want is lemmatization 
with nlp.disable_pipes('parser', 'ner'):
  #Iterate through each doc object and tag lemma, append lemma to list
  for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
    word_list = []
    for token in doc:
        word_list.append(token.lemma_)
        
    lemma_list.append(word_list)

#Make pos list a new column in dataframe
enriched_df['Lemmas'] = lemma_list
enriched_df['Lemmas'] = [' '.join(map(str, l)) for l in enriched_df['Lemmas']]

#Check lemmas
enriched_df.head()

Lemmatization can help reduce noise and refine results for researchers who are conducting keyword searches. For example, let's compare counts of the word "write" in the original token column and in the lemmatized column

In [ ]:
print(f'"Write" appears in the text tokens column ' + str(enriched_df.Text_NoStops.str.count("write").sum()) + ' times.')
print(f'"Write" appears in the lemmas column ' + str(enriched_df.Lemmas.str.count("write").sum()) + ' times.')

As expected, there are more instances of "write" in the lemmas column, as the lemmatization process has grouped inflected word forms (writing, writer) into the base word "write." For similar rasons, lemmatized data is also often used to [create topic models](https://towardsdatascience.com/topic-modelling-in-python-with-spacy-and-gensim-dc8f7748bdbf).

### b. Part of Speech Tagging

The nlp pipeline also enables the tagging of each word according to its part of speech. This code will append all parts of speech to a new DataFrame column. 


In [ ]:
#Get part of speech tags
pos_list = []

# Disable Dependency Parser, and NER since all we want is POS 
with nlp.disable_pipes('parser', 'ner'):
  #Iterate through each doc object and tag POS, append POS to list
  for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
    word_list = []
    for token in doc:
        word_list.append(token.pos_)
        
    pos_list.append(word_list)

#Make pos list a new column in DataFrame
enriched_df['POS_Tags'] = pos_list
enriched_df['POS_Tags'] = [' '.join(map(str, l)) for l in enriched_df['POS_Tags']]

#Check pos tags
enriched_df.head()

From here, you may want to get only a set of Part of Speech tags for further analysis--all of the proper nouns, for instance. 

In [ ]:
#Get specific subset of part of speech tags
propnoun_list = []

# Disable Dependency Parser, and NER since all we want is POS 
with nlp.disable_pipes('parser', 'ner'):
  #Iterate through each doc object and tag POS, append POS to list
  for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
    word_list = []
    for token in doc:
      if token.pos_ == 'PROPN':
        word_list.append(token)
        
    propnoun_list.append(word_list)

#Make pos list a new column in DataFrame
enriched_df['Proper_Nouns'] = propnoun_list
enriched_df['Proper_Nouns'] = [', '.join(map(str, l)) for l in enriched_df['Proper_Nouns']]

#Check proper noun tags
enriched_df.head()

Check out the dictionary of SpaCy POS tags [here](https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/#:~:text=Spacy%20POS%20Tags%20List,-Every%20token%20is%20assigned%20a) and feel free to test out the process of retreiving different parts of speech using the code above. 

Closely related to POS tagging is dependency parsing, wherein SpaCy identifies how different segments of a text are related to each other. Once the grammatical structure of each sentence is identified, visualizations can be created to show the connections between different words. Since we are working with large texts, our code will break down each text into sentences (spans) and then create dependency visualizers for each span


In [ ]:
#Get dependency parsing for a single doc
doc = nlp(enriched_df.Text_NoStops[0]) 
print(doc)

#Make each sentence a span to break up dependency visualizations
spans = doc.sents

#Create dependency visualizations 
displacy.render(spans, style="dep", jupyter=True)

Dependency parsing also enables the extraction of larger chunks of text, like noun phrases; let's try it out:

In [ ]:
#Get part of speech tags
np_list = []

# Disable Dependency Parser, and NER since all we want is POS 
with nlp.disable_pipes('ner'):
  #Iterate through each doc object and tag POS, append POS to list
  for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
    word_list = []
    for np in doc.noun_chunks:
      word_list.append(np)
    np_list.append(word_list)

#Make pos list a new column in DataFrame
enriched_df['Text_NounPhrases'] = np_list
enriched_df['Text_NounPhrases'] = [', '.join(map(str, l)) for l in enriched_df['Text_NounPhrases']]

#Check pos tags
enriched_df.head()

Part of speech tagging is another way to reduce dimensionality of the data and identify how language is being used at the lexico-grammatical level. Some types of further analysis, like topic modeling, may work when only certain parts of speech, [like nouns,](https://aclanthology.org/U15-1013.pdf) are retained. And as we'll explore further below, parts of speech can be used to compare patterns of language use within subsets of a corpora (e.g. texts of different genres and disciplines)

### c. Named Entity Recognition

Finally, SpaCy can tag “named entities” in your text, such as names, dates, organizations, and locations. We’ll again call the nlp pipeline on each document in the corpus and append the named entity tags to a new column. 


In [ ]:
#Get Named Entities
ner_list = []

with nlp.disable_pipes('tagger', 'parser'):
    for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
      ent_list = []
      for ent in doc.ents:
        ent_list.append(ent.label_)
      ner_list.append(ent_list)
 
enriched_df['NER_Tags'] = ner_list
enriched_df['NER_Tags'] = [' '.join(map(str, l)) for l in enriched_df['NER_Tags']]


#Check named entities
enriched_df.head()

We can add another column with the words and phrases identified as named entities.

In [ ]:
#Get Named Entitie words
ent_w_list = []

with nlp.disable_pipes('tagger', 'parser'):
    for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
        ent_w_list.append(doc.ents)

enriched_df['NER_Words'] = ent_w_list
enriched_df['NER_Words'] = [' '.join(map(str, l)) for l in enriched_df['NER_Words']]


#Check named entities
enriched_df.head()

SpaCy also allows you to visualize named entities within single texts, as follows: 

In [ ]:
#Get named entities in a single document and visualize
doc = nlp(enriched_df.Text_NoStops[0]) 

displacy.render(doc, style="ent", jupyter=True)

Named entity recognition allows researchers to take a closer look at the "real-world objects" that are present in their texts: names, persons, institutions, events, works of art, and [more](https://www.kaggle.com/code/curiousprogrammer/entity-extraction-and-classification-using-spacy). In addition to studying named entities that spaCy automatically recognizees, you can use a training dataset to update the categories or create a new entity category, as in [this example](machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/).

## 5. Analysis of Linguistic Annotations

Why are spaCy's linguistic annotations useful to researchers? Below are two examples of how researchers can use data about the MICUSP corpus, produced through spaCy, to draw conclusions about discipline and genre conventions in student academic writing. 

### a. Part of Speech Analysis

**Research Question:** Do students use certain *parts of speech* more frequently in Biology papers versus English papers that signify differences in disciplinary conventions?  

Prior research has shown that even when writing in the same genres, writers in the sciences follow different conventions than those in the humanities. Notably, academic writing in the sciences has been characterized informational, descriptive, and procedural, while that in the humanities is narrativized, evaluative, and situation-dependent (i.e. focused on discussing a particular text or prompt) [(Hardy and Römer (2013)](https://web.p.ebscohost.com/ehost/pdfviewer/pdfviewer?vid=0&sid=df763712-4f88-480f-a421-987bb35a09cd%40redis). 

By deploying spaCy on the MICUSP texts, researchers can determine whether there are any significant differences between the part-of-speech tag frequencies in English and Biology papers. For example, we might expect students writing Biology papers to use more adjectives than those in the humanities, given their focus on description. Conversely, we might suspect English papers have more verbs and verb auxiliaries, indicating a more narrative structure. We'll use spaCy to test these hypotheses as well as explore other part-of-speech count differences that could prompt further investigation. 

To start, we'll create a new DataFrame with the text filenames, disciplines, and part of speech tags. 

In [ ]:
#Create new DataFrame for analysis purposes
pos_analysis_df = enriched_df[['Filename','DISCIPLINE', 'POS_Tags']]
pos_analysis_df.head()

Before beginning the analysis, calculate the total number of part-of-speech tags in each discipline. This indicates to what extent our two sample groups are well-balanced. In this case, they're close enough (within a few thousand words) that we'll proceed with comparing the raw average frequency counts of each part of speech per discipline, though in a more thorough analysis it's recommended to normalize any frequency results using a method like term frequency-inverse document frequency [(tf-idf)](https://programminghistorian.org/en/lessons/analyzing-documents-with-tfidf).

In [ ]:
#Calculate the frequency of each POS tag as it appears in each cell in the POS_Tags column of the dataframe 
pos_analysis_df['POS_Counts'] = pos_analysis_df.POS_Tags.apply(lambda x: len(str(x).split(' ')))

#Sum the total number of each POS tags as it appears within all cells and add to DataFrame called counts 
counts = pos_analysis_df.groupby(['DISCIPLINE']).sum()

#Print counts
counts

spaCy's pipeline includes a way to count the number of each part of speech tag that appears in each document (ex. # times NOUN tag appears in a document, # times VERB tag appears, etc). This is called using ```doc.count_by(spacy.attrs.POS)``` Here's how it works on a single sentence. 

In [ ]:
doc = nlp("This is 'an' example? sentence")

print(doc.count_by(spacy.attrs.POS))

The output is a dictionary that lists the unique index of each part of speech and the number of times that part of speech has appeared in the example sentence. To associate the actual parts-of-speech associated with each index, a new dictionary can be created which replaces the index of each part of speech for its label. In the example below, it's now possible to see which parts-of-speech tags correspond to which counts. 

In [ ]:
#Store dictinoary with indexes and POS counts in a variable
num_pos = doc.count_by(spacy.attrs.POS)

dictionary = {}
#Create a new dictionary which replaces the index of each part of speech for its label (NOUN, VERB, ADJECTIVE)
for k,v in sorted(num_pos.items()):
  dictionary[doc.vocab[k].text] = v

dictionary

To get the same type of dictionary for each text in the DataFrame, a for loop can be created to nest the above process. The output will be a new DataFrame that includes the discipline of each paper and the frequency of each part of speech as appearing in that paper. If a paper does not contain a particular part-of-speech, the cell will read "NaN" (Not a Number). 

In [ ]:
#Get part of speech tags
num_list = []

# Disable Dependency Parser, and NER since all we want is POS 
with nlp.disable_pipes('parser', 'ner'):
  #Iterate through each doc object and tag POS, append POS to list
  for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
    dictionary = {}
    num_pos = doc.count_by(spacy.attrs.POS)
    for k,v in sorted(num_pos.items()):
      dictionary[doc.vocab[k].text] = v
    num_list.append(dictionary)

pos_counts = pd.DataFrame(num_list)
columns = list(pos_counts.columns)

idx = 0
new_col = pos_analysis_df['DISCIPLINE']
pos_counts.insert(loc=idx, column='DISCIPLINE', value=new_col)

pos_counts.head()

From here, it's simple to calculate the average number of each part of speech tag that appears in texts per genre by combining the ```groupby``` and ```mean``` functions. The output will be another DataFrame which shows the average part-of-speech use for each category in English and Biology.

In [ ]:
average_pos_df = pos_counts.groupby(['DISCIPLINE']).mean()
average_pos_df = average_pos_df.round(2)
average_pos_df = average_pos_df.reset_index()
average_pos_df

Now we can examine the differences between parts of speech usage per genre. As suspected, it looks like Biology student papers use slightly more adjectives (17), on average, than English student papers, while an even greater number English papers (48), on average, use more verbs. Another interesting contrast is in the "NUM" tag: almost 50 more tokens, on average, are identified as a numeral in Biology papers than in English ones. Given the conventions of scientific research, this too makes sense; studies are much more frequently quantitative, incorporating lab measurements and statistical calculations. A bar graph can make some of these differences more apparent.

In [ ]:
#Use plotly to plot proper noun use per genre
fig = px.bar(average_pos_df, x="DISCIPLINE", y=["ADJ", 'VERB', "NUM"], title="Average Part-of-Speech Usage Differences in Biology and English Student Writing", barmode='group')
fig.show()

Though admittedly a simple analysis, calculating part-of-speech frequency counts affirms prior studies which posit a correlation between lexico-grammatical features and disciplinary conventions [(Hardy and Römer (2013)](https://web.p.ebscohost.com/ehost/pdfviewer/pdfviewer?vid=0&sid=df763712-4f88-480f-a421-987bb35a09cd%40redis) and indicates an application of spaCy that can be adapted to serve other researchers' corpora and part-of-speech usage queries. spaCy also provides a "fine-grained" tag set that could aid further research--for example, looking at how Biology and English students use sub-groups of verbs with different frequencies. Fine-grain tagging can be deployed in a similar loop to those above; just instead of retrieving the ```token.pos_``` for each word, call spacy to retrieve the ```token.tag_```:

In [ ]:
#Get part of speech tags
tag_num_list = []

# Disable Dependency Parser, and NER since all we want is POS 
with nlp.disable_pipes('parser', 'ner'):
  #Iterate through each doc object and tag POS, append POS to list
  for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
    tag_dictionary = {}
    for token in doc: 
      num_tag = doc.count_by(spacy.attrs.TAG)
      for k,v in sorted(num_tag.items()):
        dictionary[doc.vocab[k].text] = v
      tag_num_list.append(tag_dictionary)

tag_counts = pd.DataFrame(num_list)
columns = list(tag_counts.columns)

idx = 0
new_col = pos_analysis_df['DISCIPLINE']
tag_counts.insert(loc=idx, column='DISCIPLINE', value=new_col)
tag_counts

From here, the same type of frequency analysis can be performed on the fine-grained tags. P 

The example here is only one of many possible applications for part-of-speech tagging. art-of-speech tagging is also useful for research questions about sentence [*intent*](https://nostarch.com/download/samples/NLP_Vasiliev_ch2.pdf); the meaning of a text changes depending on whether the past, present, or infinitive form of a particular verb is used. It's valuable for word sense disambiguation and language translation. And of course, part-of-speech tagging is a building block of named entity recogntion, the focus of the analysis below.  

### b. Named Entity Analysis

**Research Question:** Do students use certain *named entities* more frequently in different academic genres that signify differences in genre conventions?  

As with disciplines, research has shown that different genres of writing have their own conventions and expectations [(Berkenkotter and Huckin, 1995)](https://psycnet.apa.org/record/1995-97008-000). For example, explanatory genres like research papers, proposals and reports focus on analysis and explanation, whereas argumentative and critique-driven papers are driven by evaluations and arguments ([Römer and O'Donnell, 2011](https://web.s.ebscohost.com/ehost/pdfviewer/pdfviewer?vid=0&sid=0b9af0f6-d23e-47ae-90dc-e1ea9fbe606a%40redis); [Aull and Lancaster (2016)](https://journals.sagepub.com/doi/epub/10.1177/0741088318819472).

By deploying spaCy on the MICUSP texts, researchers can determine whether there are any significant differences between the named entity frequencies in papers within the seven different genres represented (Argumentative Essay, Creative Writing, Critique/Evaluation, Proposal, Report, Research Paper, and Response Paper). We may suspect that argumentative genres engage more with people or organizations--in effect, entities with or against which they are taking a stance. Conversely, perhaps dates and numbers are more prevalent in evidence-heavy genres, like research papers and proposals. We'll use spaCy to test these hypotheses. 

To start, we'll create a new DataFrame with the text filenames, genres, and named entity tags. 

In [ ]:
#Create new DataFrame for analysis purposes
ner_analysis_df = enriched_df[['Filename','PAPER TYPE', 'NER_Tags', 'NER_Words']]
ner_analysis_df.head()

Using the str.count method, we can get counts of a specific named entity used in each text. Let's get the counts of the named entities of interest here (PERSON, ORG, DATE, and CARDINAL (numbers) ) and add them as new columns of the DataFrame. 

In [ ]:
#Get the number of each type of entity in each paper
person_counts = ner_analysis_df['NER_Tags'].str.count('PERSON')
org_counts = ner_analysis_df['NER_Tags'].str.count('ORG')
date_counts = ner_analysis_df['NER_Tags'].str.count('DATE')
cardinal_counts = ner_analysis_df['NER_Tags'].str.count('CARDINAL')

#Append proper noun counts to new DataFrame 
ner_counts_df = pd.DataFrame()
ner_counts_df['Genre'] = ner_analysis_df["PAPER TYPE"]
ner_counts_df['PERSON_Counts'] = person_counts
ner_counts_df['ORG_Counts'] = org_counts
ner_counts_df['DATE_Counts'] = date_counts
ner_counts_df['CARDINAL_Counts'] = cardinal_counts

ner_counts_df.head()

From here, we can calculate the average usage of each named entity and plot across paper type.


In [ ]:
#Calculate average usage of each named entity type
average_ner_df = ner_counts_df.groupby(['Genre']).mean()
average_ner_df = average_ner_df.round(2)
average_ner_df = average_ner_df.reset_index()
average_ner_df

#Use plotly to plot proper noun use per genre
fig = px.bar(average_ner_df, x="Genre", y=["PERSON_Counts", 'ORG_Counts', "DATE_Counts", 'CARDINAL_Counts'], title="Average Part-of-Speech Usage Differences in Biology and English Student Writing", barmode='group')
fig.show()

As hypothesized, the most dates and numbers are used in description-heavy proposals and research papers, while more people are referenced in critiques and evaluations. Interestingly, organizations are most invoked in proposals. Considering that spaCy defines ORG entities as companies, organizations, and institutions, this may still make sense in the context of research proposals, which involve putting forward a research question and justifying a study. Students thus may be making references to prior scholarship and institutions which have significance to new study. To explore this phenomenon further, retrieve the words tagged as this entity in each document. 

In [ ]:
#Get Named Entitie words
date_w_list = []

for doc in nlp.pipe(enriched_df.Text_NoStops.astype('unicode').values, batch_size=100):
  ent_list = []
  for ent in doc.ents:
    if ent.label_ == 'DATE':
      ent_list.append(ent.text)
  date_w_list.append(ent_list)

ner_analysis_df['DATE_Words'] = date_w_list
ner_analysis_df['DATE_Words'] = [', '.join(map(str, l)) for l in ner_analysis_df['DATE_Words']]


#Check named entities
ner_analysis_df.head()

Now retrieve only the subset of essays that are in the proposal genre and get the top words that have been tagged as "DATE in these essays and append them to a list. The majority are standard 4-digit dates, as well as an instance of 'al'; though further analysis is certainly needed to confirm, these date entities seem to indicate citation references are occuring; this fits in with the expectations of the genre, which require references to prior scholarship to justify the student's proposed claim.

In [ ]:
#Search for only date words in proposal genre essays 
date_word_counts_df = ner_analysis_df[(ner_analysis_df == 'Proposal').any(axis=1)]

#Count the frequency of each word in these essays and append to list
date_word_frequencies = date_word_counts_df.DATE_Words.str.split(expand=True).stack().value_counts()
date_word_frequencies[:10]

Let's contrast this with the top "DATE" entities in Critique/Evaluation papers. 

In [ ]:
date_word_counts_df = ner_analysis_df[(ner_analysis_df == 'Critique/Evaluation').any(axis=1)]
date_word_frequencies = date_word_counts_df.DATE_Words.str.split(expand=True).stack().value_counts()
date_word_frequencies[:10]

Here, only three of the top dates tagged are words, and the rest are noun references to relative dates or periods. This too may indicate genre conventions such as providing context and/or centering an argument in relative space and time. Future research could analyze chains of named entities (and parts of speech) to get a better understanding of how these features work together to indicate larger rhetorical efforts. 

## 6. Download Enriched Dataset

To wrap up, download the DataFrame with the enriched versions of the text. 

In [ ]:
enriched_df.to_csv('enriched_dataset.csv') 

##Conclusions
Through this tutorial, we've gleaned more information about the grammatical makeup of a text corpus. Such information can be valuable to researchers who are seeking to understand differences between texts in their corpus - for example, *what types of named entities are most common across the corpus? How frequently are certain words used as nouns vs. objects within individual texts and corpora, and what may this reveal about the content or themes of the texts themselves?* 

SpaCy is also a helpful tool to explore texts without fully-formed research questions in mind; exploring linguistic annotations like those mentioned above can propel further questions and text-mining pipelines, like the following: 
*   [Getting Started with Topic Modeling and Mallet (Graham, Weingart, and Milligan, 2012)](https://programminghistorian.org/en/lessons/topic-modeling-and-mallet#what-is-topic-modeling-and-for-whom-is-this-useful) - Describes process of conducting topic modeling on a corpora; the SpaCy tutorial can serve as a preliminary step to clean and explore data to be used in topic modeling
*   [Sentiment Analysis for Exploratory Data Analysis (Saldaña, 2018)](https://programminghistorian.org/en/lessons/sentiment-analysis#calculate-sentiment-for-a-paragraph) - Describes how to conduct sentiment analysis using NLTK; the SpaCy tutorial provides alternative methods of pre-processing and exploration of entities that may become relevant in sentiment analysis 



## Bibliography

Aull, L. (2019). Linguistic Markers of Stance and Genre in Upper-Level Student Writing. Written Communication, 36(2), 267–295. https://doi.org/10.1177/0741088318819472

Burns, H. (2021). Crowdsourced-Data Normalization with Python and Pandas. Programming Historian, 10. https://doi.org/10.46430/phen0093

Dombrowski, Q., Gniady, T., & Kloster, D. (2019). Introduction to Jupyter Notebooks. Programming Historian, 8. https://doi.org/10.46430/phen0087

Froehlich, H. (2015). Corpus Analysis with Antconc. Programming Historian, 4. https://doi.org/10.46430/phen0043

Graham, S., Weingart, S., & Milligan, I. (2012). Getting Started with Topic Modeling and MALLET. Programming Historian, 1. https://doi.org/10.46430/phen0017

Hardy, J. A., & Römer, U. (2013). Revealing disciplinary variation in student writing: A multi-dimensional analysis of the Michigan Corpus of Upper-level Student Papers (MICUSP). Corpora, 8(2), 183–207. https://doi.org/10.3366/cor.2013.0040

Kim, S. (2018). ‘Two rules are at play when it comes to none ’: A corpus-based analysis of singular versus plural none: Most grammar books say that the number of the indefinite pronoun none depends on formality level; corpus findings show otherwise. English Today, 34(3), 50–56. https://doi.org/10.1017/S0266078417000554

Martin, F. and  Johnson, M. (2015). More Efficient Topic Modelling Through a Noun Only Approach . In Proceedings of Australasian Language Technology Association Workshop, pages 111−115. https://aclanthology.org/U15-1013.pdf 

O’Donnell, M. B., & Römer, U. (2012). From student hard drive to web corpus (part 2): The annotation and online distribution of the Michigan Corpus of Upper-level Student Papers (MICUSP). Corpora, 7(1), 1–18. https://doi.org/10.3366/cor.2012.0015

Römer, U., & O’Donnell, M. B. (2011). From student hard drive to web corpus (part 1): The design, compilation and genre classification of the Michigan Corpus of Upper-level Student Papers (MICUSP). Corpora, 6(2), 159–177. https://doi.org/10.3366/cor.2011.0011

Saldaña, Z. W. (2018). Sentiment Analysis for Exploratory Data Analysis. Programming Historian, 7. https://doi.org/10.46430/phen0079

Turkel, W. J., & Crymble, A. (2012). Counting Word Frequencies with Python. Programming Historian, 1. https://doi.org/10.46430/phen0003

Turkel, W. J., & Crymble, A. (2012). Python Introduction and Installation. Programming Historian, 1. https://doi.org/10.46430/phen0009

Wieringa, J. (2012). Intro to Beautiful Soup. Programming Historian, 1. https://doi.org/10.46430/phen0008
